Prepare the environment

In [0]:
import pandas as pd # pandas
import numpy as np # numpy

In [0]:
# Upload the files
# from google.colab import files
# uploaded = files.upload()

# Load the csv files
df_order = pd.read_csv('order_brush_order.csv', parse_dates=True)

# Use smaller datasets for faster testing
# df_order = df_order.iloc[0:25000]

In [3]:
# Create a new column for indicating the time where the 1 hour brushing period ends (even_time + 1 hour)
def brush_time_max(row):
  row['event_time_max'] = row['event_time'] + pd.to_timedelta(1, unit='h')
  return row

# Parse the event_time into a pandas.Timestamp object
df_order['event_time'] = pd.to_datetime(df_order['event_time'])

# Get the end of brushing period
df_order = df_order.apply(brush_time_max, axis='columns')
df_order

,orderid,shopid,userid,event_time,event_time_max
0,31076582227611,93950878,30530270,2019-12-27 00:23:03,2019-12-27 01:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20,2019-12-27 12:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35,2019-12-27 14:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00,2019-12-27 14:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55,2019-12-27 12:37:55
...,...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59,2019-12-29 00:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20,2019-12-28 20:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52,2019-12-28 09:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31,2019-12-28 11:14:31


In [4]:
# Gets the concentration of each shop in an hour and lists the suspicious buyers
def concentrate(row):
  # Get the the orders done within the time of event_time to even_time_max
  session = df_order[(df_order['shopid'] == row['shopid']) & (df_order['event_time'].between(row['event_time'], row['event_time_max']))]
  
  # Get a list all the buyers
  unique_buyers = session['userid'].unique()

  # Get the total count of all transactions / orders
  all_order_count = len(session.index)

  # Compute the concetrate (total number of orders within an hour / number of unique buyers)
  concentrate = all_order_count / len(unique_buyers)

  # Get the buyers with the higher propotions when the concentrate for the shop is greater than 3
  if concentrate >= 3:
    users = session.groupby('userid')['orderid'].agg(['count'])
    users = users[users['count'] == users['count'].max()]
    row['suspicious_users'] = users.index.values
  else:
    row['suspicious_users'] = np.NaN

  row['concentrate'] = concentrate

  return row

# Get the concetration and suspicious buyers
df_filtered = df_order.apply(concentrate, axis='columns')
df_filtered

,orderid,shopid,userid,event_time,event_time_max,suspicious_users,concentrate
0,31076582227611,93950878,30530270,2019-12-27 00:23:03,2019-12-27 01:23:03,NaN,1.00
1,31118059853484,156423439,46057927,2019-12-27 11:54:20,2019-12-27 12:54:20,NaN,1.00
2,31123355095755,173699291,67341739,2019-12-27 13:22:35,2019-12-27 14:22:35,NaN,1.00
3,31122059872723,63674025,149380322,2019-12-27 13:01:00,2019-12-27 14:01:00,NaN,1.00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55,2019-12-27 12:37:55,NaN,1.50
...,...,...,...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59,2019-12-29 00:17:59,NaN,1.00
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20,2019-12-28 20:07:20,NaN,1.00
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52,2019-12-28 09:17:52,NaN,2.00
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31,2019-12-28 11:14:31,NaN,1.08


In [5]:
# Create a single string containing all of the suspicious buyers from a given shopid
def brush(df):
  # Get the orders where the concentrate became equal or greater than 3
  df = df[df['concentrate'] >= 3]

  # Return 0 when no suspicious buyer is found for this shopid
  if len(df.index) == 0:
    return '0'

  # Combine the list of suspicious buyers into a single numpy array
  suspicious_buyers = []

  for index, buyers in df['suspicious_users'].items():
      suspicious_buyers = np.concatenate([buyers, suspicious_buyers])

  # Remove the duplicates
  suspicious_buyers = np.unique(suspicious_buyers)

  # Join each userid into a single string. 
  return '&'.join(str(int(x)) for x in suspicious_buyers.tolist())

df_grouped = df_filtered.groupby('shopid').apply(brush)

# Reset columns
df_grouped = df_grouped.to_frame()
df_grouped = df_grouped.reset_index()
df_grouped.columns = ['shopid', 'userid']

# Preview shopid with suspicious buyers
df_grouped[df_grouped['userid'] != '0']

,shopid,userid
40,10402,77819
57,10536,672345
111,42472,740844
114,42818,170385453
129,76934,190449497
...,...,...
17401,203531250,114282846
17960,204225676,198662175
18155,208696908,214111334
18557,210197928,52867898


In [0]:
df_grouped.to_csv('submission.csv', index=False)
from google.colab import files
files.download("submission.csv")